# Submitted by: N.PAVAN PRATEEK


# For Bipolar Factory internship Assessment.


# Date of Submission: 27rd April 2020

In [102]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article  
import csv 
import nltk
import pandas as pd
import numpy as np

# Crawling News from Times  Website

In [103]:
url = "https://timesofindia.indiatimes.com/world"
r = requests.get(url)

soup = BeautifulSoup(r.content, 'html5lib')
table = soup.findAll('a', attrs = {'class':'w_img'})

In [104]:
news=[]
for row in table: 
    if not row['href'].startswith('http'):
        news.append('https://timesofindia.indiatimes.com'+row['href'])

In [105]:
df=[]
for i in news:
    article = Article(i, language="en")
    article.download() 
    article.parse() 
    article.nlp() 
    data={}
    data['Title']=article.title
    data['Text']=article.text
    data['Summary']=article.summary
    data['Keywords']=article.keywords
    df.append(data)

In [106]:
dataset=pd.DataFrame(df)
dataset.head()

,Title,Text,Summary,Keywords
0,Kim Jong Un Health: South Korea maintains Kim ...,FILE - North Korean leader Kim Jong Un. (AP ph...,FILE - North Korean leader Kim Jong Un.\n(AP p...,"[untrue, koreas, kim, health, kims, south, mai..."
1,Covid-19: Mayor entertains residents after imp...,"Apr 27, 2020, 08:27AM IST\n\nSource: AP\n\nThe...","Apr 27, 2020, 08:27AM ISTSource: APThe videos,...","[facebook, way, entertains, imposing, try, kaw..."
2,UK PM Boris Johnson returns to work after reco...,"Apr 27, 2020, 08:40AM IST\n\nSource: Times Now...","Apr 27, 2020, 08:40AM ISTSource: Times NowBrit...","[returns, virus, work, uk, worsened, secretary..."
3,"1 officer dead, 1 wounded in Louisiana shooting","Apr 27, 2020, 08:28AM IST\n\nSource: AP\n\nPol...","Apr 27, 2020, 08:28AM ISTSource: APPolice say ...","[officer, shot, law, officers, wounded, vetera..."
4,Covid-19: Lockdown reveals fresh and clean air...,"Apr 22, 2020, 04:03PM IST\n\nSource: AP\n\nInd...","Apr 22, 2020, 04:03PM ISTSource: APIndia's ext...","[stay, workplaces, visibility, india, reveals,..."


# Model for predicting virality of news

In [107]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [118]:
FILEPATH=r"C:\Users\pavan\Downloads\crawl.csv"

In [119]:
def clean_cols(data):
    clean_col_map = {x: x.lower().strip() for x in list(data)}
    return data.rename(index=str, columns=clean_col_map)

def TrainTestSplit(X, Y, R=0, test_size=0.2):
    return train_test_split(X, Y, test_size=test_size, random_state=R)

#### I have droped these features 

In [120]:
full_data = clean_cols(pd.read_csv(FILEPATH))
train_set, test_set = train_test_split(full_data, test_size=0.20, random_state=42)

x_train = train_set.drop(['url','shares','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','abs_title_subjectivity','abs_title_sentiment_polarity'], axis=1)
y_train = train_set['shares']

x_test = test_set.drop(['url','shares', 'self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','abs_title_subjectivity','abs_title_sentiment_polarity'], axis=1)
y_test = test_set['shares']

In [121]:
clf = RandomForestRegressor(random_state=42)
clf.fit(x_train, y_train)

C:\Users\pavan\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

# Converting Crawled News

In [122]:
from nltk.tokenize import word_tokenize 
from textblob import TextBlob
def token(text):
    text=text
    return word_tokenize(text)

In [123]:
pos_words=[]
neg_words=[]
def polar(words):
    tokens=token(words)
    for i in tokens:
        analysis=TextBlob(i)
        p=analysis.sentiment.polarity
        if p>0:
            pos_words.append(i)
        if p<0:
            neg_words.append(i)
    return pos_words,neg_words

In [124]:
def rates(words):
    words=words
    pos=words[0]
    neg=words[1]
    all_words=words
    global_rate_positive_words=(len(pos)/len(all_words))/100
    global_rate_negative_words=(len(neg)/len(all_words))/100
    pol_pos=[]
    pol_neg=[]
    for i in pos:
        a=TextBlob(i)
        pol_pos.append(a.sentiment.polarity)
        avg_positive_polarity=a.sentiment.polarity
    for j in neg:
        a1=TextBlob(j)
        pol_neg.append(a1.sentiment.polarity)
        avg_negative_polarity=a1.sentiment.polarity

In [125]:
df2=[]
for i in news:
    pred_info={}
    article = Article(i, language="en")
    article.download() 
    article.parse()
    a2=TextBlob(article.text)
    polarity=a2.sentiment.polarity
    title_a=TextBlob(article.title)
    pred_info['text']=article.text
    pred_info['n_tokens_title']=len(tokenize(article.title))
    pred_info['n_tokens_content']=len(tokenize(article.text))
    pred_info['num_hrefs']=article.html.count("https://timesofindia.indiatimes.com")
    pred_info['num_imgs']=len(article.images)
    pred_info['title_subjectivity']=title_a.sentiment.subjectivity
    pred_info['title_sentiment_polarity']=title_a.sentiment.polarity
    df2.append(pred_info)

In [126]:
pred_df=pd.DataFrame(df2)
pred_test=pred_df.drop(['text'],axis=1)
pred_df.head()

,text,n_tokens_title,n_tokens_content,num_hrefs,num_imgs,title_subjectivity,title_sentiment_polarity
0,FILE - North Korean leader Kim Jong Un. (AP ph...,15,732,227,13,0.0,0.000000
1,"Apr 27, 2020, 08:27AM IST\n\nSource: AP\n\nThe...",8,190,185,9,0.0,0.000000
2,"Apr 27, 2020, 08:40AM IST\n\nSource: Times Now...",11,97,185,9,0.0,0.000000
3,"Apr 27, 2020, 08:28AM IST\n\nSource: AP\n\nPol...",9,153,185,9,0.4,-0.200000
4,"Apr 22, 2020, 04:03PM IST\n\nSource: AP\n\nInd...",10,124,185,9,0.6,0.333333


# Final Results of the Likelihood of Virality of News

In [127]:
test2=pd.DataFrame(clf.predict(pred_test),pred_df['text'])
test2.reset_index(level=0, inplace=True)
test2 = test2.rename(index=str, columns={"index": "News", 0: "Virality"})
test2

,text,Virality
0,FILE - North Korean leader Kim Jong Un. (AP ph...,35398.7
1,"Apr 27, 2020, 08:27AM IST\n\nSource: AP\n\nThe...",5050.0
2,"Apr 27, 2020, 08:40AM IST\n\nSource: Times Now...",5456.0
3,"Apr 27, 2020, 08:28AM IST\n\nSource: AP\n\nPol...",6739.6
4,"Apr 22, 2020, 04:03PM IST\n\nSource: AP\n\nInd...",6770.0
5,"Apr 22, 2020, 03:58PM IST\n\nSource: AP\n\nIra...",1877.1
6,"Apr 22, 2020, 04:04PM IST\n\nSource: AP\n\nAn ...",8010.0
7,"Apr 22, 2020, 03:57PM IST\n\nSource: AP\n\nFou...",4726.0
8,"Apr 21, 2020, 09:25AM IST\n\nSource: Times Now...",36140.0
9,"Apr 21, 2020, 03:38PM IST\n\nSource: AP\n\nA s...",6135.1
